# IBM Advanced Data Science Capstone Project
## Sentiment Analysis of Amazon Customer Reviews
### Harsh V Singh, Apr 2021

## Extract, Transform, Load (ETL)

This notebook contains the comprehensive step-by-step process used for cleaning and preparing the raw data. 

1. The data that we are using for this project is avaiable to us in the form of two csv files (train.csv/ test.csv). We will read these files into memory and then store them in parquet files with the same name. *Spark csv reader is not able to handle commas within the quoted text of the reviews. Hence, we will first read the files into Pandas dataframes and then export them into parquet files*.

2. Since the training data is quite large, we will conduct the initial data exploration and analysis on a sample set of ~10,000 rows. Once we have finalized the ETL steps, we will implement them onto the entire train and test sets.

3. As part of data exploration, we will look at the distribution of heading and review text lengths and number of words. We will also look at the most common words in the review texts, both for stopwords and other words.

4. As part of data processing, we will use the **nltk** package to remove stopwords, clean and tokenize the text, and lemmatize the token words. 

5. Our **target variable** will be a transformation of the review ratings. Ratings above 3 (i.e. 4/5) will be categorized as positive while ratings below 3 will be categorized as negative. *For the purpose of sentiment analysis, we will ignore all reviews with rating 3 as their categorization is ambiguous*.

6. Lastly, we will convert the tokenized word arrays into count-based and TFIDF-based sparse vectors which will be used as our final feature sets.

### 1. Importing required Python libraries and initializing Apache Spark environment

In [1]:
import numpy as np
import pandas as pd
import math
import csv
import time
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import seaborn as sns
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pyarrow

import string
from langdetect import detect, detect_langs

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from collections import defaultdict
from collections import Counter

nltk.download("stopwords", quiet=True)
nltk.download("punkt", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download("averaged_perceptron_tagger", quiet=True)
ENGLISH_STOP_WORDS = set(stopwords.words("english"))

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
from pyspark.sql.functions import udf, rand
conf = SparkConf().setMaster("local[*]") \
    .setAll([("spark.driver.memory", "16g"),\
            ("spark.executor.memory", "8g"), \
            ("spark.driver.maxResultSize", "16g")])
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

ETL_SAMPLE_SIZE = 10000

In [2]:
#spark.sparkContext.stop()

### 2. Reading data from CSV and storing parquet files

The data that we are using for this project is avaiable to us in the form of two csv files (train.csv/ test.csv). We will read these files into memory and then store them in parquet files with the same name. 

We will write a function called **readSparkDFFromParquet** will read the parquet files into memory as Spark dataframes. In case the parquet files are not found, this function will call another function called **savePandasDFToParquet** which reads the original csv files into Pandas dataframe and saves them as **parquet** files.  

*The reason why we need to read the csv files into a Pandas dataframe is bacause the Spark csv reader function is not able to handle commas within the quoted text of the reviews. In order to solve that, we will use the Pandas csv reader to process the data initially and then export them into parquet files*.


In [ ]:
# Function to print time taken by a particular process, given the start and end times
def getElapsedTime(startTime, endTime):
    elapsedTime = endTime - startTime
    return("Process time = %.2f seconds."%(elapsedTime))

In [ ]:
# Schema that defines the columns and datatypes of the data in the csv files
rawSchema = StructType([
    StructField("rating", IntegerType(), True),
    StructField("review_heading", StringType(), True),
    StructField("review_text", StringType(), True)
    ])

In [ ]:
# Function to save a Pandas dataframe as a parquet file
def savePandasDFToParquet(csvPath, parqPath, rawSchema, printTime=False):
    startTime = time.time()
    pandasDF = pd.read_csv(csvPath, header=None)
    pandasDF.columns = rawSchema.names
    pandasDF.to_parquet(parqPath, engine="pyarrow")
    endTime = time.time()
    if printTime:
        print(getElapsedTime(startTime=startTime, endTime=endTime))
    return


In [ ]:
# Function to read a parquet file into a Spark dataframe
# If the parquet file is not found, it will be created from the original csv
def readSparkDFFromParquet(csvPath, parqPath, rawSchema, printTime=False):
    parquetFile = Path(parqPath)
    if (parquetFile.is_file() == False):
        print("Parquet file not found... converting %s to parquet!"%(csvPath))
        savePandasDFToParquet(csvPath=csvPath, parqPath=parqPath, rawSchema=rawSchema, printTime=printTime)
    sparkDF = spark.read.parquet(parqPath)
    return (sparkDF)


We will load the train and test sets and print a few samples as well as the size of the datasets.

In [ ]:
trainRaw = readSparkDFFromParquet(csvPath="data/train.csv", parqPath="data/train.parquet", rawSchema=rawSchema, printTime=True)
testRaw = readSparkDFFromParquet(csvPath="data/test.csv", parqPath="data/test.parquet", rawSchema=rawSchema, printTime=True)
trainRaw.show(5)
print("There are %d/ %d samples in the training/ test data."%(trainRaw.count(), testRaw.count()))
print("Sample review text: %s"%(trainRaw.take(1)[0]["review_text"]))

Since the training dataset is quite large, we will conduct the data exploration and basic analysis on a sample set, whose size is defined as a global variable, **ETL_SAMPLE_SIZE**. We will convert this to a Pandas dataframe for the analysis.

In [ ]:
sampleRaw = trainRaw.orderBy(rand()).limit(ETL_SAMPLE_SIZE).toPandas()
sampleRaw.head()

### 3. Data exploration

We need to ensure that the entire training dataset consists only of **English** language reviews. We will use the **langdetect** package to check that and drop all the training data rows where language is not *en*.

In [ ]:
# Function that call the detect function in langdetect package to predict the language of a given text
def detectTextLanguage(text):
    try:
        lang = detect(text)
    except:
        lang = "error"
    return lang

langDetectUDF = udf(lambda x: detectTextLanguage(x), StringType())

In [ ]:
sampleRaw["lang"] = sampleRaw.apply(lambda x: detectTextLanguage(x["review_text"]), axis=1)
sampleRaw.drop(sampleRaw[sampleRaw["lang"] != "en"].index, inplace=True)
sampleRaw.drop(columns="lang", inplace=True)

print("There are %d samples left after dropping non-english language reviews."%(sampleRaw.shape[0]))

In [ ]:
# Function that plots multiple customized histograms given certain datasets  
def plotHistograms(datasets, titles, figTitle, figSize=(18,6), numCols=1):
    fig = plt.figure(figsize=figSize)
    sns.set_theme()
    sns.set_style("white")
    
    numRows = math.ceil(len(datasets) / numCols)
    for i in range(len(datasets)):
        fig.add_subplot(numRows, numCols, i+1)
        sns.histplot(data=datasets[i])
        plt.xlabel("")
        plt.ylabel("")
        plt.title(titles[i])
    
    fig.suptitle(figTitle)
    plt.show()

We will plot histograms in order to visualize the length of the review heading and text. Similarly, we will also look at the distribution for number of words in the headings and review texts.

In [ ]:
plotHistograms(
    datasets=[
        sampleRaw['review_heading'].str.len(),
        sampleRaw['review_text'].str.len()],
    titles=["Review Headings", "Review Text"],
    figTitle="Distribution of String Lengths (Sample Data)",
    figSize=(18,6), numCols=2
)

In [ ]:
plotHistograms(
    datasets=[
        sampleRaw['review_heading'].str.split().map(lambda x: len(x)),
        sampleRaw['review_text'].str.split().map(lambda x: len(x))
        ],
    titles=["Review Headings", "Review Text"],
    figTitle="Distribution of Word Counts (Sample Data)",
    figSize=(18,6), numCols=2
)

### 3. Data cleansing and pre-processing


In [ ]:
# Function that returns the top N most common words and their counts
def getSortedWordCounts(wordCounts, topN=0):
    sortedCounts = [[k, v] for k, v in sorted(wordCounts.items(), key=lambda item: -item[1])]
    sortedCounts = pd.DataFrame(sortedCounts, columns = ["word", "count"]) 
    if(topN > 0):
        sortedCounts = sortedCounts.head(min(topN, sortedCounts.shape[0]))
    return (sortedCounts)


In [ ]:
# Function that uses word_tokenize from the nltk package to convert text strings into word tokens
# This function also cleans the token words by removing any punctuation and only keeps words which contain alphabets 
def getWordTokensFromText(textData):
    rawTokens = word_tokenize(textData)
    cleanTokens = [w.lower().translate(str.maketrans('', '', string.punctuation)) for w in rawTokens]
    wordList = [word for word in cleanTokens if word.isalpha()]
    return (wordList)

In [ ]:
# Function that takes as input a list of words and their counts to return the most common stopwords and other words in the list
def getTopWords(wordList, stopWords, topN=25):
    stopCounts = defaultdict(int)
    otherCounts = defaultdict(int)
    for word in wordList:
        if word in stopWords:
            stopCounts[word] += 1
        else:
            otherCounts[word] += 1

    topStopWords = getSortedWordCounts(stopCounts, topN)
    topOtherWords = getSortedWordCounts(otherCounts, topN)

    return ({"stopWords": topStopWords, "otherWords": topOtherWords})

In [ ]:
# Creating a copy of the sample data to convert the text into tokenized word arrays
sampleTokenized = sampleRaw.copy(deep=True)
sampleTokenized["review_heading"] = [getWordTokensFromText(text) for text in sampleTokenized["review_heading"]]
sampleTokenized["review_text"] = [getWordTokensFromText(text) for text in sampleTokenized["review_text"]]

headingWords = sampleTokenized["review_heading"].apply(pd.Series).stack().reset_index(drop = True).to_list()
textWords = sampleTokenized["review_text"].apply(pd.Series).stack().reset_index(drop = True).to_list()

# Get the list fo most common heading/ text words (stopwords and others)
topHeadingWords = getTopWords(wordList=headingWords, stopWords=ENGLISH_STOP_WORDS, topN=25)
topTextWords = getTopWords(wordList=textWords, stopWords=ENGLISH_STOP_WORDS, topN=25)

print("There are %d words in the review texts of %d samples."%(len(textWords), sampleTokenized.shape[0]))


In [ ]:
# Function that plots multiple custom horizontal bar plots
def plotBars(datasets, titles, x, y, figTitle, figSize=(12,6), numCols=1):
    fig = plt.figure(figsize=figSize)
    sns.set_theme()
    sns.set_style("white")
    
    numRows = math.ceil(len(datasets) / numCols)
    for i in range(len(datasets)):
        fig.add_subplot(numRows, numCols, i+1)
        sns.barplot(data=datasets[i], x=y, y=x)
        plt.xlabel("")
        plt.ylabel("")
        plt.title(titles[i])
    fig.suptitle(figTitle)
    plt.show()

In [ ]:
plotBars(
    datasets=[topHeadingWords["stopWords"], topHeadingWords["otherWords"], topTextWords["stopWords"], topTextWords["otherWords"]], 
    titles=["Headings - Stop Words", "Headings - Other Words", "Text - Stop Words", "Text - Other Words"],
    x="word", y="count", 
    figTitle="Count of Top Words in Headings and Review Texts (Sample Data)", 
    figSize=(20,12), numCols=2)

Next, we will process the tokenized data by combining the token words in the headings and review texts into a single column called **review_content**. We will also categorize all the data into positive/ negative sentiment reviews. All reviews with ratings more than 3 will be categorized as positive while all reviews with ratings less than 3 will be categorized as negative.

*We will drop all the rows where review rating is 3 as they are ambiguous in terms of the customers sentiment.*

In [ ]:
sampleProcessed = sampleTokenized.copy()
sampleProcessed["review_content"] = sampleProcessed["review_heading"] + sampleProcessed["review_text"]
sampleProcessed.loc[sampleProcessed["rating"] < 3, "review_sentiment"] = 0
sampleProcessed.loc[sampleProcessed["rating"] > 3, "review_sentiment"] = 1
sampleProcessed.drop(columns=["review_heading", "review_text", "rating"], inplace=True)
sampleProcessed.dropna(axis=0, inplace=True)
sampleProcessed.head()

In [ ]:
# Function to remove stopwords from a given array of words
def removeStopWordsFromText(textData, stopWords):
    relevantText = [word for word in textData if word not in stopWords]
    return (relevantText)

### 4. Dimensionality reduction by removing stop words and Lemmatization

In order to reduce the dimensionality of the feature set and also reduce the noise in the training data, we will remove all the stop words from the review_content. 

In [ ]:
sampleProcessed["review_content"] = [removeStopWordsFromText(text, ENGLISH_STOP_WORDS) for text in sampleProcessed["review_content"]]
sampleProcessed.head()

Now that we have a list of relevant words from each review, we will convert these words into their respective *lemmas*. This process is known as **Lemmatisation** in linguistics and is the process of grouping together the inflected forms of a word so they can be analysed as a single item. For example, variations of the word run such as ran, running, runs, etc. will all be replaced with the dictionary form of the word, i.e., run. This will further help in reducing the dimensionality of the feature set.  

In [ ]:
# Function that returns the NOUN form of any given word, using wordnet data from the nltk package
def getWordnetPos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tagDictionary = {
      "J": wordnet.ADJ,
      "N": wordnet.NOUN,
      "V": wordnet.VERB,
      "R": wordnet.ADV
      }
  return (tagDictionary.get(tag, wordnet.NOUN))

# Function that returns the lemmatized version of the text, i.e. replacing each word with its lemma
def getLemmatizedText(textData, lemmatizer):
  lemText = [lemmatizer.lemmatize(word, getWordnetPos(word)) for word in textData]
  return (lemText)

In [ ]:
startTime = time.time()
lemmatizer = nltk.stem.WordNetLemmatizer()
sampleProcessed["review_content"] = [getLemmatizedText(text, lemmatizer) for text in sampleProcessed["review_content"]]
endTime = time.time()
print(getElapsedTime(startTime=startTime, endTime=endTime))
sampleProcessed.head()


Finally, we can use the CountVectorizer function of the **scikit-learn** package to convert each review from a tokenized, lemmatized word array into a sparse vector containing the counts of each word in the review. 

In [ ]:
countVect = CountVectorizer()
reviewCounts = countVect.fit_transform(sampleProcessed["review_content"].apply(" ".join))
print("Review content is transformed into a %s with %s elements."%(type(reviewCounts), reviewCounts.shape, ))

### 5. Saving the preprocessed training and test data

We will then split this sample data into training and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reviewCounts, sampleProcessed["review_sentiment"], test_size=0.3, random_state=123)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test)
print("MultinomialNB Accuracy:", metrics.accuracy_score(y_test, predicted))

In [ ]:
tfVect = TfidfVectorizer()
reviewTF = tfVect.fit_transform(sampleProcessed["review_content"].apply(" ".join))
print("Review texts are transformed into a %s with %s elements."%(type(reviewTF), reviewTF.shape, ))

In [ ]:
startTime = time.time()
trainClean = trainRaw.withColumn("lang", langDetectFunc("review_text"))
trainClean = trainClean.filter(trainDF["lang"] == "en")
trainClean = trainClean.drop("lang")
trainClean.show(5)
endTime = time.time()
print(getElapsedTime(startTime=startTime, endTime=endTime))
#print("There are %d samples left after dropping non-english language reviews."%(trainClean.count()))